In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# To Avoid GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

#### 1. How to save and load model weights
#### 2. Save and loading entire model (Serializing model)
####   - Saves weights
####  - Model architecture
####   - Training Configuration (model.compile())
####   - Optimizer and states

In [2]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units


    def build(self, input_shape):
        self.w = self.add_weight(  # w is a trainable weight
        name='w',
            shape=( input_shape[-1], self.units),  # shape of the weight matrix
            initializer='random_normal',     # initializer of the weight matrix
            trainable=True,                 # trainable or not
        )

        self.b = self.add_weight(  # b is a trainable weight
            name='b', shape=( self.units),  # shape of the bias vector
            initializer='zeros',        # initializer of the bias vector
            trainable=True,             # trainable or not
        )



    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)

    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)



model = MyModel()
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)
print(model.summary())
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/2
938/938 - 3s - loss: 0.4020 - accuracy: 0.8913
Epoch 2/2
938/938 - 2s - loss: 0.1854 - accuracy: 0.9470
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  50240     
_________________________________________________________________
dense_1 (Dense)              multiple                  650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
157/157 - 0s - loss: 0.1515 - accuracy: 0.9557


[0.15149042010307312, 0.9556999802589417]

In [3]:
model.save("saved_model/")

INFO:tensorflow:Assets written to: saved_model/assets


In [6]:
# SavedModel format or HDF5 format
# model3 = MyModel()
model = keras.models.load_model('saved_model/')
print(model.summary())


Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  50240     
_________________________________________________________________
dense_1 (Dense)              multiple                  650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
